In [14]:
import numpy as np
import pandas as pd
from category_encoders          import *
from sklearn.base               import BaseEstimator
from sklearn.compose            import *
from sklearn.decomposition      import PCA
from sklearn.ensemble           import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute             import *
from sklearn.linear_model       import LogisticRegression
from sklearn.metrics            import confusion_matrix, classification_report, f1_score
from sklearn.model_selection    import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline           import Pipeline
from sklearn.preprocessing      import StandardScaler
import warnings

In [15]:
# Filter to remove warnings for RandomizedSearchCV so the results are easier to read.
warnings.filterwarnings("ignore")

In [16]:
# Loading data into pandas dataframe.
combine_df = pd.read_csv('red_and_white_wine.csv')

# Target Transformation: treating "great" wine as wine that has a quality score greater than or equal to 7. All else will be treated as "poor" quality
combine_df['quality'] = combine_df.apply(lambda row: 1 if row['quality'] >= 7 else 0, axis = 1)

# Seperating y and X
y = combine_df['quality']
X = combine_df[['fixed acidity', 
                'volatile acidity', 
                'citric acid', 
                'residual sugar',
                'chlorides', 
                'free sulfur dioxide', 
                'total sulfur dioxide', 
                'density',
                'pH', 
                'sulphates', 
                'alcohol', 
                'red wine']]

In [17]:
# Class to use for RandomizedSearchCV to tune hyperparameters
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass

In [18]:
# Setting random seed
np.random.seed(21)

random_state = np.random.RandomState(42)


# Splitting the dataset into training (80%) and testing (20%)
X_training, X_test, y_training, y_test = train_test_split(X, 
                                                          y, 
                                                          test_size=.2,
                                                          random_state=random_state)

# Splitting the training dataset into a train set (80%) and a validation set (20%)
X_train, X_valid, y_train, y_valid = train_test_split(X_training, 
                                                      y_training, 
                                                      test_size=.2,
                                                      random_state=random_state)


# Storing the categorical variable (whether it's red wine or white wine).
cat_cols = ['red wine']

# Storing the continuous variables.
cot_cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
            'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
            'pH', 'sulphates', 'alcohol']

# No missing data in dataset, adding imputers for missing test data to build a more robust system. 

# Using mode for this imputer it makes the most sense for categorical variables.
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True)),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))
                    ])

# Using median for this imputer as it makes the most sense for continuous variables.
cont_pipe = Pipeline([('scaler', StandardScaler()),
                      ('imputer', SimpleImputer(strategy='median', add_indicator=True))
                     ])

# ColumnTransformer for column-by-column preprocessing.
preprocessing = ColumnTransformer([('categorical', cat_pipe, cat_cols),
                                   ('continuous', cont_pipe, cot_cols)
                                  ])

# Preparing pipeline for RandomizedSearchCV.
pipe = Pipeline([('prep', preprocessing),
                 # Principal Component Analysis.
                 ('pca', PCA()),
                 # Dummy placeholder so we can test different algorithms with different hyperparameters.
                 ('clf', DummyEstimator())])

# Hyperparameter search across features, algorithms, and hyperparameters
search_space = [
                # Seeing if dimension reduction is useful for our model.
                {'pca__n_components': [0,1,2,3,4,5]},
                
                {'clf': [LogisticRegression()],
                 # Searching through penalty as it is used to specify the norm used in penalization. Some penalties work better for certain datasets.
                 'clf__penalty': ['l1', 'l2', 'elasticnet', 'none'],
                 # C is the inverse of regularizaiton strength. Has an effect on the lambda regulator.
                 'clf__C': np.logspace(0, 4, 10),
                 # The weights associated with the classes. Can help with imbalanced datasets.
                 'clf__class_weight': ['balanced','None'],
                 # Determines which algorithm is used for optimization. Different algorithms are useful for different datasets.
                 'clf__solver' : ['newton-cg','lbfgs', 'liblinear', 'sag', 'saga']},
    
                {'clf': [RandomForestClassifier()],
                 # The function to measure the quality of a split for each decision node. Can help with computation speed and accuracy.
                 'clf__criterion': ['gini', 'entropy'],
                 # The number of trees used in the forest. Can help with generalizing the model.
                 'clf__n_estimators': [50,100,150,200],
                 # The number of features to consider for each split. Can help with generalization.
                 'clf__max_features': ['auto','sqrt','log2'],
                 # The maximum depth of a tree. Can also help with generalization.
                 'clf__max_depth': [2,3,4,5,6,7,8,9,10,20,50,100],
                 # The weights associated with the classes. Can help with imbalanced datasets.
                 'clf__class_weight' : [None, 'balanced','balanced_subsample']},
                
                {'clf': [ExtraTreesClassifier()],
                 # The function to measure the quality of a split for each decision node. Can help with computation speed and accuracy.
                 'clf__criterion': ['gini', 'entropy'],
                 # The number of trees used in the forest. Can help with generalizing the model.
                 'clf__n_estimators': [50,100,150,200],
                 # The number of features to consider for each split. Can help with generalization.
                 'clf__max_features': ['auto','sqrt','log2'],
                 # The maximum depth of a tree. Can also help with generalization.
                 'clf__max_depth': [2,3,4,5,6,7,8,9,10,20,50,100],
                 # The weights associated with the classes. Can help with imbalanced datasets.
                 'clf__class_weight' : [None, 'balanced','balanced_subsample']}]

# Using Randomized Search as it can often lead to better results in a quicker time frame than grid search.
clf_algos_rand = RandomizedSearchCV(random_state=random_state,
                                    estimator=pipe, 
                                    param_distributions=search_space,
                                    # Using 300 candidates for our model search.
                                    n_iter=200,
                                    # Using StratifiedKFold to help with data imbalance.
                                    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state),
                                    # Using n_jobs as one as it helps with the reproducibility of my results.
                                    n_jobs=1,
                                    verbose=1)

best_model = clf_algos_rand.fit(X_train, y_train)

# Plugging in the best model from the RandomizedSearchCV
pipe = Pipeline([('prep', preprocessing),
                 ('rf',  best_model.best_estimator_.get_params()['clf'])])

pipe.fit(X_training, y_training)
y_pred = pipe.predict(X_test)
f1_test  = f1_score(y_test, y_pred, average='weighted')


print(f"F1 metric: {f1_test:.2%}\n")

print('Confusion Matrix:')
print(pd.DataFrame(confusion_matrix(y_true=y_test, y_pred=y_pred)))


print('\nClassification Report:')
print(classification_report(y_test, y_pred))

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2000 out of 2000 | elapsed: 11.6min finished


F1 metric: 88.48%

Confusion Matrix:
      0    1
0  1030   18
1   118  134

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      1048
           1       0.88      0.53      0.66       252

    accuracy                           0.90      1300
   macro avg       0.89      0.76      0.80      1300
weighted avg       0.89      0.90      0.88      1300



In [22]:
# PCA was not used in our final model pipeline
print(best_model.best_estimator_.get_params()['pca'])

PCA()


In [23]:
# Visually displayed single, best final model and final model parameters
print(best_model.best_estimator_.get_params()['clf'])

ExtraTreesClassifier(class_weight='balanced', criterion='entropy',
                     max_depth=100, max_features='log2', n_estimators=50)


## Chosen Model:

From my experience the three best models I have used in classification were Logistic Regression, Random Forest Classifier, Extra Trees Classifier. I chose these three models as in the past, they have performed extremely well on binary classification problems. I used a pipeline to search through hyperparameters for each model, and Extra Trees Classifier was chosen to be the best possible model of the three. Extra Trees Classifier fits a number of randomized decision trees on various subsamples of the datatset.

## Hyperparameters for best model:

The best model had the following hyperparameters: class_weight='balanced', criterion='entropy', max_depth=100, max_features='log2', n_estimators=50. 

## Comments on hyperparameters:

The class_weight hyperparameter refers to the weights associated with each class. In this case, "balanced" means the weights on each class are automatically adjusted to be inversely proportional to class frequencies based on the values of the target variable.

The critierion hyperparameter 'entropy' is important as the critierion is used to measure the quality of the split. "Entropy" means that our best model is calculating the quality of a split based off of the entropy score instead of the Gini value. Entropy is a more complex computation and sometimes gives results that are slightly better while Gini is computationally less expensive.

The max_depth hyperparameter is important as it indicates the maximum depth that an individual tree can reach. For ExtraTreesClassfier, max depth restricts how many splits a individual tree can make, keeping it from getting too specific to the training set, thus possibly increasing the model's generality.

The max_features hyperparameter designates the number of features that will be used while looking for the best split for each node. For our model, the best model uses the log base 2 of the total number of features to pick a subsample of features that will be chosen from at a specific decision node. Restricting the available features when searching for a split can help improve the generality of the model.

The n_estimators hyperparameter designates the number of trees in our ExtraTreesClassifier. The number of trees can increase the accuracy greatly (initially). However, at a specific point the increase of accuracy with the addition of one tree decreases greatly, thus it is important to find a balance. In this case, it seems the n_estimators = 50 seems to work as a good balance for our model.

In [ ]:
F1 metric: 88.48%

Confusion Matrix:
      0    1
0  1030   18
1   118  134

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      1048
           1       0.88      0.53      0.66       252

    accuracy                           0.90      1300
   macro avg       0.89      0.76      0.80      1300
weighted avg       0.89      0.90      0.88      1300

#### Evaluation Metrics

For this model I decided to use two evaluation metrics: the F1 score, the confusion matrix, and the classification report. I used the F1 score as it gives insight on the classification problems a model may be facing. The F1 score equally weighs precision and recall for our model. The F1 score for our model is high (88.48%), thus it has a nice balance between precision and recall.

However, to get a clearer picture, I decided to include the confusion matrix and the classification report. From the confusion matrix two things stand out, our False Negative rate for "great" wines is high (118) when compared to our True Positive rate (134), which indicates that our model was having a hard time predicting if a wine had a quaity score of 7 or above. The second thing that stands out is that our model has a great False Negative rate, only predicting a few (18) low quality wines as higher quality. From the classification report we see that the precision and recall score for low quality wines (wines with quality scores below 7) was very high (precision=0.90, recall=0.98), while the precision and recall scores for high quality wine (wines with quality scores 7 and above) was lower (precision=0.88, recall=0.53), especially the recall score.

All in all, our model performed well and had a good balance between the recall and precision score. Our metrics tell us that our model is relatively good on predicting the quality of wine, especially predicting if wine is of poor quality. However, there is still some room for improvement in predicting "great" wine.

In [21]:
# Display all pipelines steps and all non-default hyperparameters
print(pipe)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['red wine']),
                                                 ('continuous',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('imputer',
                                                                   S

## Summary of findings

I believe that I came up with a decent model to predict the quality of red/white wine based off its chemical composition. I think that while there is some room for improvement, especially in the case of having better precision and recall for "great" wines, this model is a great start and does a decent job as can be seen from the F1 score and the confusion matrix/classification report. I think things worked well because I chose three models that work well for this kind of classification problem, and I think the target engineering that I performed on the "quality" variable was extremely beneficial. I believe that it was smart to switch this into a binary prediction problem, and that the distinction between "poor" wine and "great" wine is accurate, as most people would state that anything below 7/10 should be seen as average or worse than average.

This project matters because the wine industry has expanded exponentially over the past two decades. As more people become interested in wine, the importance of identifying good wine increases and become more lucrative.

My next steps would be to try different more powerful algorithms. I would want to try a XGBoost model and compare it against my own, to see if there is any significant change in the recall and precision score for "great" wines. I may also want to include more data from other wine raters, as a diverse range of raters would help make my model more robust for the business enviroment as currently we only have ratings from the original data. 